In [3]:
!pip install scikit-learn
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 22.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
import nltk
from nltk.corpus import stopwords
import json
import glob
import re

nltk.download('stopwords')

def remove_stops(text, stops):
    # Normalize case and remove punctuation
    text = text.lower().translate(str.maketrans("", "", string.punctuation))
    
    # Split text into words and filter out stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stops]
    
    # Join words back into a single string
    final_text = " ".join(filtered_words)
    
    # Remove digits
    final_text = ''.join([i for i in final_text if not i.isdigit()])
    
    return final_text

def clean_plots(docs):
    # Load stopwords and convert to set for faster lookup
    stops = set(stopwords.words("english"))
    
    # Process each document using remove_stops
    final_docs = [remove_stops(doc, stops) for doc in docs]
    
    return final_docs

plots_json = pd.read_json("./results/plots.json")
plots = plots_json['plot']
cleaned_plots = clean_plots(plots)
ids = plots_json['id']

vectorizer = TfidfVectorizer(
                                lowercase=True,
                                max_features=100,
                                max_df=0.8,
                                min_df=5,
                                ngram_range = (1,3),
                                stop_words = "english"

                            )

vectors = vectorizer.fit_transform(cleaned_plots)

feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

for description in denselist:
    x=0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x=x+1
    all_keywords.append(keywords)

true_k = 20

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=100, n_init=1)

model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

cluster_labels = model.labels_

# Map cluster labels to original IDs
id_to_cluster = pd.DataFrame({'id': ids, 'cluster': cluster_labels})

# Optionally, save this mapping to a CSV file
id_to_cluster.to_csv('results/id_to_cluster.csv', index=False)

cluster_terms = pd.DataFrame(columns=['Cluster', 'Top Terms'])

rows = []
for i in range(true_k):
    top_terms = ', '.join([terms[ind] for ind in order_centroids[i, :10]])
    # cluster_terms = cluster_terms.add({'Cluster': f'Cluster {i}', 'Top Terms': top_terms})
    rows.append({'Cluster': f'Cluster {i}', 'Top Terms': top_terms})

cluster_terms = pd.concat([cluster_terms, pd.DataFrame(rows)], ignore_index=True)

cluster_terms.to_csv('results/cluster_terms.csv', index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
